# **SPAM SMS DETECTION**

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive/')
data = pd.read_csv("/content/drive/MyDrive/spam.csv", encoding='latin1')

data.columns

Mounted at /content/drive/


Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
import chardet  # For detecting the file encoding
import nltk

# Download NLTK stopwords data
nltk.download('stopwords')

# Detect the encoding of the CSV file
with open('/content/drive/MyDrive/spam.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read())
encoding = result['encoding']

# Load your dataset with the detected encoding
data = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding=encoding)
data = data[['v1', 'v2']]  # Select only relevant columns

# Preprocess the text data
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

data['v2'] = data['v2'].apply(preprocess_text)

# Label encoding for 'v1' column (spam or legitimate)
label_encoder = LabelEncoder()
data['v1'] = label_encoder.fit_transform(data['v1'])

# Split the dataset into training and testing sets
X = data['v2']
y = data['v1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the max_features
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train a Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel='linear')  # You can try different kernels
svm_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test_tfidf)

# Evaluate the model and print the classification report
report = classification_report(y_test, y_pred, target_names=['Legitimate', 'Spam'])
print("Classification Report:")
print(report)

# Print the predictions
for i in range(len(X_test)):
    message = X_test.iloc[i]
    label = "Spam" if y_pred[i] == 1 else "Legitimate"
    print(f"Message: {message}\nPredicted Label: {label}\n")

# Print the overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Classification Report:
              precision    recall  f1-score   support

  Legitimate       0.98      1.00      0.99       965
        Spam       0.98      0.87      0.92       150

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Message: funni fact nobodi teach volcano 2 erupt, tsunami 2 arise, hurrican 2 sway aroundn 1 teach hw 2 choos wife natur disast happen
Predicted Label: Legitimate

Message: sent score sopha secondari applic schools. think think applying, research cost also. contact joke ogunrinde, school one less expens one
Predicted Label: Legitimate

Message: know someon know fanci you. call 09058097218 find who. pobox 6, ls15hb 150p
Predicted Label: Spam

Message: promis get soon can. text morn let know made ok.
Predicted Label: Legitimate

Message: congratul ur award either å£500 cd gift voucher & free entri 2 å£100 weekli draw txt music 87066 tnc www.lde